<a href="https://colab.research.google.com/github/jorgemunozl/vla-test/blob/main/pi05-test/eight_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Eight_Test

We are ready for train guys

In [ ]:
from huggingface_hub import login
login()

In [ ]:
# Clone


Cloning into 'XHUMAN'...
remote: Enumerating objects: 2024, done.
remote: Counting objects: 100% (230/230), done.
remote: Compressing objects: 100% (151/151), done.
remote: Total 2024 (delta 137), reused 145 (delta 76), pack-reused 1794 (from 2)
Receiving objects: 100% (2024/2024), 7.80 MiB | 14.58 MiB/s, done.
Resolving deltas: 100% (1280/1280), done.


In [3]:
%cd XHUMAN

/content/XHUMAN


In [4]:
!uv pip install -e .[pi]

Using Python 3.12.12 environment at: /usr
Resolved 215 packages in 466ms
Prepared 1 package in 798ms
Uninstalled 1 package in 0.41ms
Installed 1 package in 0.96ms
 ~ xhuman==0.1.0 (from file:///content/XHUMAN)


In [ ]:
!git status

On branch main
Your branch is up to date with 'origin/main'.

nothing to commit, working tree clean


In [ ]:
!git restore .

In [5]:
DS_ID = "NONHUMAN-RESEARCH/TEST_RECORD_ANNOTATIONS"
PRETRAINED_PATH = "lerobot/pi05_base"

In [6]:
from lerobot.datasets.lerobot_dataset import LeRobotDataset

dataset = LeRobotDataset(DS_ID)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

Only run when debugging.

In [12]:
import sys
import importlib

# Polyfill 'imp' for Python 3.12 compatibility
if "imp" not in sys.modules:
    import types
    imp = types.ModuleType("imp")
    imp.reload = importlib.reload
    sys.modules["imp"] = imp

%load_ext autoreload
%autoreload 2

In [7]:
from xhuman.policies.pi05.configuration_pi05 import  PI05Config
device = "cuda"
policy_config = PI05Config(
        # pretrained_path=PRETRAINED_PATH,  # No Set pretrained path so factory loads weights
        device=device,
    )

In [8]:
from xhuman.policies.factory import make_xhuman_policy
policy = make_xhuman_policy(
        cfg=policy_config,
        ds_meta=dataset.meta,
    )

/usr/local/lib/python3.12/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

Monkey fix the transformers dependencies problem

In [10]:
from transformers.models.gemma import modeling_gemma

def fixed_extra_repr(self):
    if hasattr(self, "weight"):
        repr_str = f"{tuple(self.weight.shape)}, eps={self.eps}"
    else:
        repr_str = f"eps={self.eps}"
    if getattr(self, "dense", None) is not None:
        repr_str += f", adaptive=True, cond_dim={self.cond_dim}"
    return repr_str

modeling_gemma.GemmaRMSNorm.extra_repr = fixed_extra_repr

In [9]:
from xhuman.policies.pi05.processor_pi05 import (
    make_pi05_pre_post_processors_ki,
)

preprocessor, postprocessor = make_pi05_pre_post_processors_ki(
    config=policy_config,
    dataset_stats=dataset.meta.stats,
)

The way on how we obtain a batch on the training is way different from the training.

In [ ]:
import torch
# Prepare images, images_masks, tokens, masks, actions
batch = preprocessor(dataset[0])

tokens_id = "observation.language.tokens"
mask_id = "observation.language.mask"

images, img_masks = policy._preprocess_images(batch)
tokens, masks = batch[tokens_id], batch[mask_id]
noise = None
time = None
actions = policy.prepare_action(batch)

In [ ]:
        tokens = tokens.clone()
        # Embed actions and observations.
        if noise is None:
            noise = policy.model.sample_noise(actions.shape, actions.device)

        if time is None:
            time = policy.model.sample_time(actions.shape[0], actions.device)

        # actions shape: (B, 1, action_dim)
        actions_clone = actions.clone().to("cpu")

        # discrete_actions list of ints: (B, discrete_action_seq_len)
        discrete_actions = policy.model.fast_tokenizer(actions_clone)
        discrete_actions = (policy.model.paligemma_with_expert.act_tokens_to_paligemma_tokens(discrete_actions))  # noqa: E501

        tensors_discrete_actions = torch.tensor(
            discrete_actions, device=actions.device)

        discrete_action_seq_len = tensors_discrete_actions.shape[1]

In [ ]:
        indices = (tokens == 0).nonzero(as_tuple=True)
        if indices[1].numel() > 0:
            first_pad_idx = indices[1][0].item()
        else:
            first_pad_idx = tokens.shape[1]

        # token_loss_mask shape: (B, seq_len)
        token_loss_mask = torch.zeros_like(
            tokens, dtype=torch.bool, device=tokens.device
        )
        token_loss_mask[:, first_pad_idx:first_pad_idx + discrete_action_seq_len] = True  # noqa: E501

        # Replace discrete actions in tokens
        tokens[:, first_pad_idx:first_pad_idx + discrete_action_seq_len] = tensors_discrete_actions  # noqa: E501

        discrete_action_masks = torch.ones_like(
            tensors_discrete_actions, dtype=torch.bool, device=masks.device
        )

In [ ]:
        masks[:, first_pad_idx:first_pad_idx + discrete_action_seq_len] = discrete_action_masks  # noqa: E501

        prefix_embs, prefix_pad_masks, prefix_att_masks = policy.model.embed_prefix(
            images, img_masks, tokens, masks,
        )

In [ ]:

        # Clone prefix_att_masks before modifying since it's a view from .expand()
        prefix_att_masks = prefix_att_masks.clone()
        # prefix_att_masks structure:
        # [image_embs..., original_lang_tokens, discrete_actions]
        num_image_embs = prefix_embs.shape[1] - tokens.shape[1]

        discrete_action_start_idx = num_image_embs + first_pad_idx
        discrete_action_end_idx = (discrete_action_start_idx + discrete_action_seq_len)  # noqa: E501

        # Set att_mask=1 for discrete action tokens (causal masking)
        prefix_att_masks[:, discrete_action_start_idx:discrete_action_end_idx] = 1  # noqa: E501

        time_expanded = time[:, None, None]
        x_t = time_expanded * noise + (1 - time_expanded) * actions
        u_t = noise - actions

In [ ]:
        suffix_embs, suffix_pad_masks, suffix_att_masks, adarms_cond = policy.model.embed_suffix(  # noqa: E501
            x_t, time,
        )

        q_proj_dtype = (
            policy.model.paligemma_with_expert.paligemma.language_model.layers[0]
            .self_attn.q_proj.weight.dtype
        )

        if q_proj_dtype == torch.bfloat16:
            suffix_embs = suffix_embs.to(dtype=torch.bfloat16)
            prefix_embs = prefix_embs.to(dtype=torch.bfloat16)

        pad_masks = torch.cat([prefix_pad_masks, suffix_pad_masks], dim=1)
        att_masks = torch.cat([prefix_att_masks, suffix_att_masks], dim=1)

In [ ]:

        att_2d_masks = make_att_2d_masks(pad_masks, att_masks)

        suffix_start = prefix_pad_masks.shape[1]

        # Ensure flow matching doesn't depend on discrete actions.
        att_2d_masks[:, suffix_start:, discrete_action_start_idx:discrete_action_end_idx] = False  # noqa: E501

        position_ids = torch.cumsum(pad_masks, dim=1) - 1

        att_2d_masks_4d = policy.model._prepare_attention_masks_4d(att_2d_masks)

In [ ]:
        def forward_func(prefix_embs,
                         suffix_embs,
                         att_2d_masks_4d,
                         position_ids,
                         adarms_cond,
                         ):
            # Returns both prefix_out and suffix_out for dual loss computation
            (prefix_out, suffix_out), _ = policy.model.paligemma_with_expert.forward(
                attention_mask=att_2d_masks_4d,
                position_ids=position_ids,
                past_key_values=None,
                inputs_embeds=[prefix_embs, suffix_embs],
                use_cache=False,
                adarms_cond=[None, adarms_cond],
            )
            return prefix_out, suffix_out

        prefix_out, suffix_out = policy.model._apply_checkpoint(
            forward_func, prefix_embs, suffix_embs,
            att_2d_masks_4d, position_ids, adarms_cond
        )

        suffix_out = suffix_out[:, -policy.model.config.chunk_size:]
        suffix_out = suffix_out.to(dtype=torch.float32)


In [ ]:
import torch.nn.functional as F

In [ ]:
        def action_out_proj_func(suffix_out):
            return policy.model.action_out_proj(suffix_out)

        v_t = policy.model._apply_checkpoint(action_out_proj_func, suffix_out)

        # v_t shape: [B, chunk_size, action_dim]
        flow_matching_loss = F.mse_loss(u_t, v_t, reduction="none")
        flow_matching_loss = flow_matching_loss.mean(dim=(1, 2))

        # prefix_out shape: [B, prefix_seq_len, hidden_dim]
        def lm_head_func(prefix_out):
            return policy.model.paligemma_with_expert.paligemma.lm_head(prefix_out)

        # logits shape: [B, prefix_seq_len, vocab_size]
        logits = policy.model._apply_checkpoint(lm_head_func, prefix_out)

        num_image_embs = prefix_embs.shape[1] - tokens.shape[1]
        lang_logits = logits[:, num_image_embs:, :]

        # shift_logits shape: [B, lang_seq_len-1, vocab_size]
        shift_logits = lang_logits[:, :-1, :].contiguous()
        # shift_labels shape: [B, lang_seq_len-1]
        shift_labels = tokens[:, 1:].contiguous()

        # shift_mask shape: [B, lang_seq_len-1]
        shift_mask = token_loss_mask[:, 1:].contiguous()

In [ ]:
        # flat_logits shape: [B*(lang_seq_len-1), vocab_size]
        flat_logits = shift_logits.view(-1, shift_logits.size(-1))
        # flat_labels shape: [B*(lang_seq_len-1)]
        flat_labels = shift_labels.view(-1)
        flat_mask = shift_mask.view(-1)

        # ce_loss_per_token shape: [B*(lang_seq_len-1)]
        ce_loss_per_token = F.cross_entropy(
            flat_logits, flat_labels, reduction='none'
        )

        # masked_ce shape: [B, lang_seq_len-1]
        masked_ce = (ce_loss_per_token * flat_mask).view(tokens.shape[0], -1)  # noqa: E501
        # ce_loss shape: [B]
        ce_loss = masked_ce.sum(dim=1) / (shift_mask.sum(dim=1) + 1e-8)

In [ ]:

ce_loss, flow_matching_loss

In [ ]:
!git diff

In [ ]:
!git restore .

In [ ]:
!git status

On branch main
Your branch is up to date with 'origin/main'.

nothing to commit, working tree clean
